In [1]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00


In [2]:
# --- Imports ---
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
import numpy as np
import evaluate
import os

# --- Ensure output directories exist ---
os.makedirs("./models/news_classifier", exist_ok=True)
os.makedirs("./logs", exist_ok=True)

# --- Load dataset ---
dataset = load_dataset("ag_news")

# --- Load tokenizer ---
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

# --- Load model ---
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=4  # AG News has 4 classes
)

# --- Load metrics ---
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)["accuracy"]
    f1_score = f1.compute(predictions=preds, references=labels, average="weighted")["f1"]
    return {"accuracy": acc, "f1": f1_score}

# --- Training arguments ---
training_args = TrainingArguments(
    output_dir="./models/news_classifier",
    save_strategy="steps",
    save_steps=1000,
    eval_strategy="steps",    # <-- fixed (was eval_strategy)
    eval_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    push_to_hub=True,
    hub_model_id="Zeeshan506/news-classifier",
    hub_token=os.getenv("HF_WRITE_TOKEN")       # safer than userdata
)

# --- Define trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

# --- Train (resume if checkpoint exists) ---
trainer.train(resume_from_checkpoint=True)

# --- Save final model ---
trainer.save_model("./models/news_classifier")
tokenizer.save_pretrained("./models/news_classifier")
print("✅ Training finished and model saved successfully!")